In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PageRank").getOrCreate()
sc = spark.sparkContext
print(spark.version)


4.0.0


In [7]:
lines = sc.textFile("question2.txt")
links = lines.map(lambda x: x.split(":")).map(lambda x: (int(x[0]), [int(i) for i in eval(x[1])]))
links = links.cache()
links.take(5)

[(0, [94, 53, 69, 76, 14, 49, 75, 66, 6]),
 (1, [27, 45, 50, 93, 4, 31, 58, 86]),
 (2, [85, 5, 21, 92, 74]),
 (3, [62, 24, 13, 49, 14, 82]),
 (4, [5, 59, 88, 49, 13, 51, 55, 73, 91])]

In [8]:
links = links.partitionBy(4).cache()
ranks = links.mapValues(lambda _: 1.0)

for iteration in range(10):
    contribs = links.join(ranks).flatMap(
        lambda node_neighbors_rank: [
            (neighbor, node_neighbors_rank[1][1] / len(node_neighbors_rank[1][0]))
            for neighbor in node_neighbors_rank[1][0]
        ]
    )
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)

output = ranks.collect()
for (node, rank) in output[:10]:
    print(f"Node {node} has rank: {rank:.4f}")



Node 76 has rank: 1.3465
Node 88 has rank: 0.8742
Node 72 has rank: 0.9569
Node 96 has rank: 0.3415
Node 52 has rank: 0.8323
Node 80 has rank: 1.0348
Node 32 has rank: 0.1982
Node 24 has rank: 1.2106
Node 0 has rank: 0.9732
Node 28 has rank: 0.8270


In [9]:
max_node = max(output, key=lambda x: x[1])
min_node = min(output, key=lambda x: x[1])

print(f"Highest rank: Node {max_node[0]} with rank {max_node[1]:.4f}")
print(f"Lowest rank: Node {min_node[0]} with rank {min_node[1]:.4f}")

Highest rank: Node 86 with rank 2.8297
Lowest rank: Node 47 with rank 0.1745
